In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chi2, poisson, ttest_ind
import pandas as pd

class PoissonFlowSimulator:
    def __init__(self, student_number):
        self.N = student_number  # Student number in group list
        self.T1 = self.N
        self.T2 = self.N + 100
        self.lambda1 = (self.N + 8) / (self.N + 24)
        self.lambda2 = (self.N + 9) / (self.N + 25)
        self.lambda_total = self.lambda1 + self.lambda2
        
    def generate_exponential_random(self, lambda_param):
        """Generate exponential random variable using inverse transform method"""
        u = np.random.uniform(0, 1)
        return -np.log(u) / lambda_param
    
    def generate_poisson_flow(self, lambda_param, T1, T2):
        """Generate Poisson flow events on interval [T1, T2]"""
        events = []
        current_time = T1
        
        while current_time <= T2:
            interval = self.generate_exponential_random(lambda_param)
            current_time += interval
            if current_time <= T2:
                events.append(current_time)
        
        return np.array(events)
    
    def combine_flows(self, flow1, flow2):
        """Combine two flows by merging and sorting event times"""
        combined = np.concatenate([flow1, flow2])
        return np.sort(combined)
    
    def plot_flows(self, flow1, flow2, combined_flow, title="Poisson Flows"):
        """Visualize the flows on timeline"""
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 8))
        
        # Flow 1
        ax1.eventplot(flow1, orientation='horizontal', colors=['red'])
        ax1.set_title(f'Flow X1 (λ={self.lambda1:.3f})')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Events')
        
        # Flow 2
        ax2.eventplot(flow2, orientation='horizontal', colors=['blue'])
        ax2.set_title(f'Flow X2 (λ={self.lambda2:.3f})')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Events')
        
        # Combined flow
        ax3.eventplot(combined_flow, orientation='horizontal', colors=['green'])
        ax3.set_title(f'Combined Flow X1+X2 (λ={self.lambda_total:.3f})')
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Events')
        
        plt.tight_layout()
        plt.show()
    
    def calculate_statistics(self, flow, delta_t, lambda_theoretical=None):
        """Calculate statistical characteristics for a flow"""
        # Count events in delta_t intervals
        num_intervals = 25
        interval_counts = []
        
        for i in range(num_intervals):
            start = self.T1 + i * delta_t
            end = self.T1 + (i + 1) * delta_t
            count = np.sum((flow >= start) & (flow < end))
            interval_counts.append(count)
        
        interval_counts = np.array(interval_counts)
        
        # Basic statistics
        mean_count = np.mean(interval_counts)
        var_count = np.var(interval_counts)
        
        # Estimate lambda from data
        lambda_estimated = mean_count / delta_t
        
        # Theoretical lambda (if provided)
        if lambda_theoretical is None:
            lambda_theoretical = lambda_estimated
        
        return {
            'interval_counts': interval_counts,
            'mean_count': mean_count,
            'variance_count': var_count,
            'lambda_estimated': lambda_estimated,
            'lambda_theoretical': lambda_theoretical,
            'delta_t': delta_t
        }
    
    def chi_square_test(self, stats_dict, alpha=0.05):
        """Perform chi-square test for Poisson distribution"""
        counts = stats_dict['interval_counts']
        lambda_est = stats_dict['lambda_estimated']
        delta_t = stats_dict['delta_t']
        n = len(counts)
        
        # Get unique values and their frequencies
        unique, frequencies = np.unique(counts, return_counts=True)
        
        # Calculate theoretical probabilities
        theoretical_probs = []
        theoretical_freqs = []
        
        for k in unique:
            prob = poisson.pmf(k, lambda_est * delta_t)
            theoretical_probs.append(prob)
            theoretical_freqs.append(prob * n)
        
        # Chi-square test
        chi2_stat = 0
        for i in range(len(unique)):
            if theoretical_freqs[i] > 0:  # Avoid division by zero
                chi2_stat += (frequencies[i] - theoretical_freqs[i])**2 / theoretical_freqs[i]
        
        # Degrees of freedom: number of bins - 1 - number of estimated parameters
        df = len(unique) - 1 - 1  # -1 for estimated lambda
        chi2_critical = chi2.ppf(1 - alpha, df)
        
        return {
            'chi2_statistic': chi2_stat,
            'chi2_critical': chi2_critical,
            'degrees_freedom': df,
            'p_value': 1 - chi2.cdf(chi2_stat, df),
            'reject_null': chi2_stat > chi2_critical
        }
    
    def student_t_test(self, stats1, stats2, alpha=0.05):
        """Perform Student's t-test for homogeneity"""
        counts1 = stats1['interval_counts']
        counts2 = stats2['interval_counts']
        
        # Two-sample t-test
        t_stat, p_value = ttest_ind(counts1, counts2)
        
        # Manual calculation (as described in lecture)
        n1, n2 = len(counts1), len(counts2)
        mean1, mean2 = np.mean(counts1), np.mean(counts2)
        var1, var2 = np.var(counts1, ddof=1), np.var(counts2, ddof=1)
        
        # Pooled standard deviation
        pooled_var = ((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2)
        t_manual = (mean1 - mean2) / np.sqrt(pooled_var * (1/n1 + 1/n2))
        
        # Critical value
        df = n1 + n2 - 2
        t_critical = stats.t.ppf(1 - alpha/2, df)
        
        return {
            't_statistic': t_stat,
            't_manual': t_manual,
            't_critical': t_critical,
            'p_value': p_value,
            'reject_null': abs(t_stat) > t_critical
        }

def main():
    # Initialize simulator with student number (example: student number 5)
    simulator = PoissonFlowSimulator(student_number=5)
    
    print("=== LAB WORK 2: POISSON FLOW SIMULATION ===")
    print(f"Student number: {simulator.N}")
    print(f"Time interval: [{simulator.T1}, {simulator.T2}]")
    print(f"λ1 = {simulator.lambda1:.4f}")
    print(f"λ2 = {simulator.lambda2:.4f}")
    print(f"λ_total (theoretical) = {simulator.lambda_total:.4f}")
    print()
    
    # Generate flows
    flow1 = simulator.generate_poisson_flow(simulator.lambda1, simulator.T1, simulator.T2)
    flow2 = simulator.generate_poisson_flow(simulator.lambda2, simulator.T1, simulator.T2)
    flow_theoretical = simulator.generate_poisson_flow(simulator.lambda_total, simulator.T1, simulator.T2)
    flow_practical = simulator.combine_flows(flow1, flow2)
    
    print(f"Flow X1: {len(flow1)} events")
    print(f"Flow X2: {len(flow2)} events")
    print(f"Theoretical combined flow: {len(flow_theoretical)} events")
    print(f"Practical combined flow: {len(flow_practical)} events")
    print()
    
    # Plot flows
    simulator.plot_flows(flow1, flow2, flow_practical)
    
    # Calculate statistics
    delta_t = (simulator.T2 - simulator.T1) / 25
    
    stats_x1 = simulator.calculate_statistics(flow1, delta_t, simulator.lambda1)
    stats_x2 = simulator.calculate_statistics(flow2, delta_t, simulator.lambda2)
    stats_theoretical = simulator.calculate_statistics(flow_theoretical, delta_t, simulator.lambda_total)
    stats_practical = simulator.calculate_statistics(flow_practical, delta_t, simulator.lambda_total)
    
    # Display statistical results
    print("=== STATISTICAL CHARACTERISTICS ===")
    flows_data = [
        ("X1", stats_x1, simulator.lambda1),
        ("X2", stats_x2, simulator.lambda2),
        ("X_theoretical", stats_theoretical, simulator.lambda_total),
        ("X_practical", stats_practical, simulator.lambda_total)
    ]
    
    results = []
    for name, stats, lambda_true in flows_data:
        chi2_result = simulator.chi_square_test(stats)
        
        result = {
            'Flow': name,
            'λ_true': lambda_true,
            'λ_estimated': stats['lambda_estimated'],
            'Mean_count': stats['mean_count'],
            'Variance_count': stats['variance_count'],
            'Chi2_stat': chi2_result['chi2_statistic'],
            'Chi2_critical': chi2_result['chi2_critical'],
            'Poisson_hypothesis': 'Not rejected' if not chi2_result['reject_null'] else 'Rejected'
        }
        results.append(result)
    
    df_results = pd.DataFrame(results)
    print(df_results.to_string(index=False))
    print()
    
    # Compare theoretical and practical combined flows
    print("=== COMPARISON OF COMBINED FLOWS ===")
    t_test_result = simulator.student_t_test(stats_theoretical, stats_practical)
    
    print(f"T-test statistic: {t_test_result['t_statistic']:.4f}")
    print(f"T-critical value: {t_test_result['t_critical']:.4f}")
    print(f"P-value: {t_test_result['p_value']:.4f}")
    print(f"Hypothesis of homogeneity: {'Not rejected' if not t_test_result['reject_null'] else 'Rejected'}")
    print()
    
    # Verify the additive property
    print("=== VERIFICATION OF ADDITIVE PROPERTY ===")
    lambda_sum_estimated = stats_x1['lambda_estimated'] + stats_x2['lambda_estimated']
    lambda_practical_estimated = stats_practical['lambda_estimated']
    
    print(f"λ1_estimated + λ2_estimated = {lambda_sum_estimated:.4f}")
    print(f"λ_practical_estimated = {lambda_practical_estimated:.4f}")
    print(f"λ_theoretical = {simulator.lambda_total:.4f}")
    print()
    
    relative_error = abs(lambda_sum_estimated - lambda_practical_estimated) / lambda_practical_estimated * 100
    print(f"Relative error between sum and practical: {relative_error:.2f}%")
    
    # Mathematical calculations demonstration
    print("\n=== MATHEMATICAL CALCULATIONS ===")
    print("1. Exponential distribution generation:")
    print("   u ~ U(0,1), then interval = -ln(u)/λ")
    print("   This comes from solving: u = 1 - exp(-λt) for t")
    print("   => t = -ln(1-u)/λ ≈ -ln(u)/λ (since 1-u has same distribution as u)")
    print()
    
    print("2. Poisson process properties:")
    print("   P(X(t) = k) = (λt)^k * exp(-λt) / k!")
    print("   E[X(t)] = λt, Var[X(t)] = λt")
    print("   For Δt = 1, E[X] = λ, Var[X] = λ")
    print()
    
    print("3. Additive property proof:")
    print("   If X1 ~ Poisson(λ1) and X2 ~ Poisson(λ2) are independent,")
    print("   then X1 + X2 ~ Poisson(λ1 + λ2)")
    print("   This is verified by our simulation results.")

if __name__ == "__main__":
    main()